In [20]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10


#### Tento kód načítava dataset CIFAR-10, konvertuje obrázky na stupne grayscale, normalizuje hodnoty pixelov medzi -1 a 1 a rozdeľuje dataset na trénovaciu a validačnú časť. Obrázky v stupňoch grayscale sú preformátované na 4D tenzory s jedným kanálom, a farebné obrázky sú preformátované na 4D tenzory s tromi kanálmi.


In [22]:
from tensorflow import keras
from tensorflow.keras import layers


# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert images to grayscale
x_train_gray = np.dot(x_train, [0.299, 0.587, 0.114])
x_test_gray = np.dot(x_test, [0.299, 0.587, 0.114])

# Normalize pixel values between -1 and 1
x_train_gray = (x_train_gray / 255.0) * 2 - 1
x_test_gray = (x_test_gray / 255.0) * 2 - 1

# Reshape grayscale images to 4D tensors with a single channel
x_train_gray = x_train_gray.reshape(x_train_gray.shape[0], 32, 32, 1)
x_test_gray = x_test_gray.reshape(x_test_gray.shape[0], 32, 32, 1)

# Reshape color images to 4D tensors with 3 channels
x_train_color = x_train.reshape(x_train.shape[0], 32, 32, 3)
x_test_color = x_test.reshape(x_test.shape[0], 32, 32, 3)

AttributeError: EagerTensor object has no attribute 'astype'. 
        If you are looking for numpy-related methods, please run the following:
        from tensorflow.python.ops.numpy_ops import np_config
        np_config.enable_numpy_behavior()
      

In [26]:
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, LeakyReLU, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam


# Define generator network architecture
generator = keras.Sequential(
    [
        layers.Input(shape=(32, 32, 1)),
        layers.Conv2D(32, kernel_size=3, padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size=3, strides=2, padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2DTranspose(32, kernel_size=3, padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2DTranspose(3, kernel_size=3, padding="same", activation="tanh"),
    ],
    name="generator",
)

# Define discriminator network architecture
discriminator = keras.Sequential(
    [
        layers.Input(shape=(32, 32, 3)),
        layers.Conv2D(64, kernel_size=3, strides=2, padding="same", activation="relu"),
        layers.Dropout(0.4),
        layers.Conv2D(128, kernel_size=3, strides=2, padding="same", activation="relu"),
        layers.Dropout(0.4),
        layers.Conv2D(256, kernel_size=3, strides=2, padding="same", activation="relu"),
        layers.Dropout(0.4),
        layers.Flatten(),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)

# Define combined model architecture
discriminator.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.5))
generator.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.5))
discriminator.trainable = False
gan_input = keras.Input(shape=(32, 32, 1))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output, name="gan")
gan.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.5))



In [35]:
epochs = 50
batch_size = 128
sample_interval = 10
num_batches = len(x_train_gray)// batch_size
print(num_batches)

351


In [36]:
def sample_images(epoch):
    # Generate images from random noise
    noise = np.random.normal(0, 1, (16, 32, 32, 1))
    generated_images = generator.predict(noise)

    # Rescale pixel values between 0 and 1
    generated_images = 0.5 * generated_images + 0.5

    # Plot generated images
    fig, axs = plt.subplots(4, 4)
    count = 0
    for i in range(4):
        for j in range(4):
            axs[i,j].imshow(generated_images[count, :, :, :])
            axs[i,j].axis('off')
            count += 1
    plt.savefig(f"{epoch}.png")
    plt.close()

In [48]:
for epoch in range(epochs):
    for batch in range(num_batches):
        # Select a random batch of grayscale images
        real_images = x_train_gray[np.random.randint(0, x_train_gray.shape[0], batch_size)]
        print(len(real_images))
        # Generate a batch of color images from the grayscale images
        generated_images = generator.predict(real_images)
        print(len(generated_images))
        # Concatenate the real and generated images into a single batch
        #TODO error s axis
        x = np.concatenate([real_images,generated_images])
        # x = np.concatenate([generated_images],axis=3)
        # Label the real and generated images
        y_real = np.ones((batch_size, 1))
        y_fake = np.zeros((batch_size, 1))
        y = np.concatenate([y_real, y_fake])
        print(len(x),len(y))
        # Train the discriminator on the real and generated images
        discriminator_loss = discriminator.train_on_batch(x, y)

        # Train the generator to fool the discriminator
        noise = np.random.normal(0, 1, (batch_size, 32, 32, 1))
        generator_loss = gan.train_on_batch(noise, y_real)

    # Print the epoch number and losses
    print(f"Epoch {epoch+1}, Discriminator Loss: {discriminator_loss}, Generator Loss: {generator_loss}")

    # Save sample images
    if epoch % sample_interval == 0:
        sample_images(epoch)

128
4/4 [==============================] - 0s 17ms/step
128
128 256


ValueError: Data cardinality is ambiguous:
  x sizes: 128
  y sizes: 256
Make sure all arrays contain the same number of samples.

### Trénovavanie GAN modelu

In [ ]:
train(epochs, batch_size, sample_interval)